In [1]:
import pandas as pd
import numpy as np
import researchpy as rp
import seaborn as sns
import os
import ast

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C

In [2]:
def get_class (model):
    if isinstance(model, str):
        model = '{' +'}'.join('{'.join(model.split('{')[1:]).split('}')[0:1]) + '}'
        m = ast.literal_eval(model)
        if 'classifier:__choice__' in m:
            classifier = m['classifier:__choice__']
        else:
            classifier = ''
        return classifier
    else:
        return ''


def parse_tpot(directory):
    result = pd.DataFrame(columns=['dataset', 'accuracy', 'model', 'precision', 'recall', 'f1score', 'time_budget', 'methods'])
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                method = file.split('.')[0].split('_')[2]
                time_budget = file.split('.')[0].split('_')[3]
                run = []
                sub_result = pd.read_csv(os.path.join(subdir, file))
                sub_result.rename(columns={'Unnamed: 0': 'dataset'}, inplace = True)
                run.append(sub_result[['dataset', 'accuracy_1', 'model_1', 'precision_1', 'recall_1', 'f1score_1']])
                run.append(sub_result[['dataset', 'accuracy_2', 'model_2', 'precision_2', 'recall_2', 'f1score_2']])
                run.append(sub_result[['dataset', 'accuracy_3', 'model_3', 'precision_3', 'recall_3', 'f1score_3']])
                for i in range(3):
                    run[i].rename(columns={'accuracy_'+str(i+1): 'accuracy',
                                           'model_'+str(i+1): 'model',
                                           'precision_'+str(i+1): 'precision',
                                           'recall_'+str(i+1): 'recall',
                                           'f1score_'+str(i+1): 'f1score'}, inplace=True)
                    run[i]['methods'] = str(method)
                    run[i]['time_budget'] = int(time_budget)
                    result = pd.concat([result, run[i]], axis=0, sort=True, ignore_index=True)
    result.model = result.model.apply(get_class)
    result = result[~np.isnan(result.f1score)]
    return result
#parse_tpot(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/")

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

,accuracy,dataset,f1score,methods,model,precision,recall,time_budget
0,0.949153,AP_Breast_Lung,0.911765,3C,,0.911765,0.911765,10
4,0.947368,audiology,0.918919,3C,,0.944444,0.894737,10
5,0.700000,dataset_31_credit-g,0.788732,3C,,0.782123,0.795455,10
7,0.900807,gina_agnostic,0.892500,3C,,0.915385,0.870732,10
8,0.965974,hiva_agnostic,0.333333,3C,,0.692308,0.219512,10
...,...,...,...,...,...,...,...,...
891,0.597500,wine-quality-red,0.597500,SVC,,0.597500,0.597500,60
895,0.395026,eye_movements,0.395026,SVC,,0.395026,0.395026,60
897,0.645570,solar-flare_1,0.645570,SVC,,0.645570,0.645570,60
898,0.166667,umistfacescropped,0.166667,SVC,,0.166667,0.166667,60


In [3]:
df = parse_tpot(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("default", "fc")
df.methods = df.methods.replace("SVC", "1c")
df.methods = df.methods.replace("DT", "1c")
df.methods = df.methods.replace("RF", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df=df_binary
df.shape

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(343, 4)

In [4]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,343.0,0.72191,0.29389,0.015869,0.690698,0.753122


In [5]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,105,0.726433,0.290676,0.028367,0.670834,0.782032
30,112,0.719853,0.299978,0.028345,0.664296,0.775409
60,126,0.719970,0.293397,0.026138,0.668740,0.771200


In [6]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,190,0.635285,0.331348,0.024039,0.588170,0.682401
3C,69,0.831838,0.189870,0.022858,0.787037,0.876639
fc,84,0.827549,0.194506,0.021222,0.785953,0.869144


In [7]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                       
10          1c       57  0.648255  0.325113  0.043062   0.563853  0.732657
            3C       23  0.832530  0.204386  0.042618   0.749000  0.916060
            fc       25  0.807070  0.221663  0.044333   0.720178  0.893962
30          1c       62  0.630576  0.339170  0.043075   0.546149  0.715002
            3C       21  0.825612  0.197536  0.043106   0.741124  0.910099
            fc       29  0.834136  0.196677  0.036522   0.762553  0.905720
60          1c       71  0.628986  0.333819  0.039617   0.551337  0.706636
            3C       25  0.836432  0.176854  0.035371   0.767105  0.905758
            fc       30  0.838246  0.172466  0.031488   0.776530  0.899962

In [8]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 8, 334) =  5.121, p =  0.0000


In [9]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     5.121
Date:                Fri, 01 Nov 2019   Prob (F-statistic):           4.95e-06
Time:                        22:11:55   Log-Likelihood:                -46.333
No. Observations:                 343   AIC:                             110.7
Df Residuals:                     334   BIC:                             145.2
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                 0.6483      0.037     17.437      0.000       0.575       0.721
C(time_budget)[T.30]                     -0.0177      0.052     -0.343      0.732      -0.119       0.084
C(time_budget)[T.60]                     -0.0193      0.050     -0.386      0.700      -0.117       0.079
C(methods)[T.3C]                          0.1843      0.069      2.658      0.008       0.048       0.321
C(methods)[T.fc]                          0.1588      0.067      2.359      0.019       0.026       0.291
C(time_budget)[T.30]:C(methods)[T.3C]     0.0108      0.099      0.109      0.914      -0.184       0.206
C(time_budget)[T.60]:C(methods)[T.3C]     0.0232      0.095      0.243      0.808      -0.164       0.210
C(time_budget)[T.30]:C(methods)[T.fc]     0.0447      0.092      0.485      0.628      -0.137       0.226
C(time_budget)[T.60]:C(methods)[T.fc]     0.0504      0.091      0.555      0.579      -0.128       0.229
==============================================================================
Omnibus:                       51.094   Durbin-Watson:                   2.187
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.426
Skew:                          -1.115   Prob(JB):                     3.09e-16
Kurtosis:                       3.163   Cond. No.                         12.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.001045,2.0,0.006632,9.933903e-01
C(methods),3.194879,2.0,20.277772,4.880336e-09
C(time_budget):C(methods),0.029264,4.0,0.092868,9.846753e-01
Residual,26.311808,334.0,NaN,NaN


In [11]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 4, 338) =  10.259, p =  0.0000


In [12]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     10.26
Date:                Fri, 01 Nov 2019   Prob (F-statistic):           7.51e-08
Time:                        22:11:55   Log-Likelihood:                -46.524
No. Observations:                 343   AIC:                             103.0
Df Residuals:                     338   BIC:                             122.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.6376      0.031     20.855      0.000       0.577       0.698
C(time_budget)[T.30]    -0.0044      0.038     -0.116      0.908      -0.079       0.070
C(time_budget)[T.60]    -0.0024      0.037     -0.065      0.948      -0.075       0.070
C(methods)[T.3C]         0.1964      0.039      5.004      0.000       0.119       0.274
C(methods)[T.fc]         0.1923      0.037      5.257      0.000       0.120       0.264
==============================================================================
Omnibus:                       51.392   Durbin-Watson:                   2.185
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.949
Skew:                          -1.119   Prob(JB):                     2.38e-16
Kurtosis:                       3.173   Cond. No.                         4.17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.001045,2.0,0.006704,9.933188e-01
C(methods),3.194879,2.0,20.497822,3.960405e-09
Residual,26.341072,338.0,NaN,NaN


In [14]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.001045,0.000522,2.0,0.006704,9.933188e-01,0.000035,-0.005228
C(methods),3.194879,1.597440,2.0,20.497822,3.960405e-09,0.108165,0.102618
Residual,26.341072,0.077932,338.0,NaN,NaN,NaN,NaN


In [15]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    1c     3C   0.1966 0.001  0.1045 0.2887   True
    1c     fc   0.1923 0.001  0.1064 0.2781   True
    3C     fc  -0.0043   0.9 -0.1108 0.1022  False
--------------------------------------------------


In [16]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
    10     30  -0.0066   0.9 -0.1008 0.0877  False
    10     60  -0.0065   0.9 -0.0981 0.0852  False
    30     60   0.0001   0.9   -0.09 0.0902  False
--------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C

In [17]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

NameError: name 'pandas' is not defined

In [ ]:
rp.summary_cont(df['f1score'])

In [ ]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

In [ ]:
rp.summary_cont(df.groupby(['methods']))['f1score']

In [ ]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

In [ ]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

In [ ]:
model.summary()

In [ ]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

In [ ]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

In [ ]:
model2.summary()

In [ ]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

In [ ]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

In [ ]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C 
# Comparing FC (60 min) with 3C(30)

In [ ]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "1c")
df.methods = df.methods.replace("['decision_tree']", "1c")
df.methods = df.methods.replace("['random_forest']", "1c")
df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.drop(df[(df.methods=='1c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

In [ ]:
rp.summary_cont(df['f1score'])

In [ ]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

In [ ]:
rp.summary_cont(df.groupby(['methods']))['f1score']

In [ ]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

In [ ]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

In [ ]:
model.summary()

In [ ]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

In [ ]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

In [ ]:
model2.summary()

In [ ]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

In [ ]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

In [ ]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

# Wilcoxon signed-rank test

In [ ]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
df.shape

In [ ]:
# Wilcoxon signed-rank test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon
# seed the random number generator
seed(1)
# generate two independent samples
data1 = df.f1score[df.methods=='3c']
data2 = df.f1score[df.methods=='fc']
# compare samples
stat, p = wilcoxon(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

In [ ]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
df.shape

In [ ]:
# Wilcoxon signed-rank test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon
# seed the random number generator
seed(1)
# generate two independent samples
data1 = df.f1score[df.methods=='3c']
data2 = df.f1score[df.methods=='fc']
# compare samples
stat, p = wilcoxon(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')